<a href="https://colab.research.google.com/github/Wency-WangC/Nice-Little-Tools/blob/main/converter_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📅 曆法轉換助手 | Lunar Calendar CSV Formatter
### 🧭 使用步驟 Steps

1. 按下 Ctrl+F9 或點選左下方▶執行程式碼區塊。  
   Press Ctrl+F9 or press ▶ on the bottom left to run the code.
2. 稍等約30秒后，根據指引，选择你想使用的数据上传方式完成上传。  
   Wait for 30 seconds, then upload your data.
3. 系統將自動補欄、簡轉繁，並下載CSV檔。之后请根据指引将文件上传至中研院网站。  
   The tool will auto-complete missing fields, convert Simplified to Traditional Chinese, and download a UTF-8 CSV file for upload to Sinocal. Please proceed to submit the downloaded file to Sinocal for result.

In [ ]:
# @title
### Lunar Date Converter with Dual Input Modes for Colab
# 安装依赖
!pip install -q opencc-python-reimplemented pandas openpyxl ipywidgets

from google.colab import files
import pandas as pd
import io, os
from opencc import OpenCC
import re
import ipywidgets as widgets
from IPython.display import display, clear_output, FileLink

# 简体转繁体引擎
cc = OpenCC('s2t')

# 全局存储列表
entries = []

# 校验函数
def validate_entry(rec):
    """
    按照给定规则校验单条记录，返回错误信息或 None
    规则：
    1. 日存在时，必须同时输入月和年；月存在时，必须输入年。
    2. 如果没有年号，则必须同时填写国号和帝号；如果没有国号和帝号，则必须有年号。
    3. 月格式必须为1-12阿拉伯数字或“闰”/“閏”+数字1-12。
    4. 年和日必须为阿拉伯数字。
    """
    guo, di, era = rec['国号'], rec['帝号'], rec['年号']
    year, month, day = rec['年'], rec['月'], rec['日']
    # 规则4: 年/日数字检查
    if year and not year.isdigit():
        return '年必须为阿拉伯数字。'
    if day and not day.isdigit():
        return '日必须为阿拉伯数字。'
    # 规则1: 层级关系
    if day and not (month and year):
        return '输入了“日”，请同时输入“月”和“年”。'
    if month and not year:
        return '输入了“月”，请同时输入“年”。'
    # 规则2: 年号 vs 国号/帝号 二选一
    if not era:
        if not (guo or di):
            return '请填写“年号”，或者同时填写“国号”和“帝号”。'
        else:
          if not (guo and di):
            return '未填写“年号”，请填写“国号”和“帝号”。'
    # 规则3: 月格式
    if month and not re.match(r'^(?:闰|閏)?(?:[1-9]|1[0-2])$', month):
        return '月格式不正确，应为1-12数字或闰1-12。'
    return None

# 导出并提供下载链接

def convert_and_download(df: pd.DataFrame, filename: str = "output_input.csv"):
    for col in ["国号", "帝号"]:
        if col not in df.columns:
            df[col] = ""
    df = df[["国号", "帝号", "年号", "年", "月", "日"]].fillna("")
    df = df.astype(str).apply(lambda col: col.map(lambda x: cc.convert(x)))
    df.to_csv(filename, index=False, header=False, encoding="utf-8")
    # 自动触发下载
    files.download(filename)
    # 同时显示下载链接，便于在回调中查看
    display(FileLink(filename, result_html_prefix='下载链接：'))
    print(f"✅ 导出并下载：{filename}")

# Excel 上传分支
def upload_excel_ui():
    clear_output()
    display(widgets.HTML('''
  <h2>📄 Prepare Your Excel File</h2>
  <p>
  新建 Excel 文檔，並輸入你想轉換的所有中曆日期資料。Excel 文件可含表頭（每欄的名稱，例如「年號」「年」「月」「日」等）。每一筆日期資料為一行，請依序填入下列六列：
  </p>
  <ul>
  <li>國號 | Dynasty</li>
  <li>帝號 | Emperor</li>
  <li>年號 | Reign Title</li>
  <li>年 | Year</li>
  <li>月 | Month</li>
  <li>日 | Day</li>
  </ul>
  <p>
  ▶ 範例Excel下載 |
<a
  href="https://github.com/Wency-WangC/Nice-Little-Tools/raw/main/sample_data.xlsx"
  download="sample_data.xlsx"
>
  <strong>Download Sample Excel</strong>
</a>
  '''))
    tips_html = '''
  <p>● 由於多數年號為歷史上唯一<sup>*</sup>，若不清楚「國號／帝號」，可留空並僅填年號。<br>
  &nbsp;&nbsp;Since most reign titles are historically unique<sup>*</sup>, if you're unsure about the dynasty or emperor, you may leave those fields blank and provide only the reign title.<br>
  ● Excel內可使用干支記載年日（例如「甲辰年」「乙丑日」等）。<br>
  &nbsp;&nbsp;The system accepts sexagenary cycles expression of year and day.<br>
  ● 日期可省略：省略「日」表示該月初一；省略「月日」表示該年首日；三項皆省略表示年號起始日。<br>
  &nbsp;&nbsp;Date input can be abbreviated: omitting the day means the first day of the month; omitting month and day means the first day of the year; omitting year, month, and day indicates the first day of the reign period.</p>
  <br><p>
<small><sup>*</sup> 約有 38 種年號名稱重複，請參考 Sinocal 說明文件。<br>
<sup>*</sup> Approximately 38 reign titles are non-unique; please refer to the Sinocal documentation for clarification.</small>
</p>
'''
    accordion = widgets.Accordion(children=[widgets.HTML(tips_html)])
    accordion.set_title(0, 'Show Tips')
    accordion.selected_index = None  # 默认折叠
    display(accordion)  # 展示可折叠提示
    display(widgets.HTML('''<h2>请在下面上传你的Excel文件</h2>'''))
    uploaded = files.upload()
    fname = list(uploaded.keys())[0]
    df = pd.read_excel(io.BytesIO(uploaded[fname]), dtype=str)
    print("✅ 上传并读取完成，前几行：")
    display(df.head())
    convert_and_download(df, f"{os.path.splitext(fname)[0]}_input.csv")

# 手动输入分支：单条录入
def manual_input_ui():
    clear_output()
    display(widgets.HTML('<b>说明：</b>请逐条输入各字段，然后点击“下一条”以添加，完成后点击“提交并处理”生成并下载 CSV。'))
    field_guo = widgets.Text(description='国号：', placeholder='例如明、清')
    field_di = widgets.Text(description='帝号：', placeholder='例如成祖、高宗')
    field_era = widgets.Text(description='年号：', placeholder='例如康熙、成化')
    field_year = widgets.Text(description='年：', placeholder='阿拉伯数字，例如5')
    field_month = widgets.Text(description='月：', placeholder='1-12或闰1-12')
    field_day = widgets.Text(description='日：', placeholder='阿拉伯数字，例如14')

    next_btn = widgets.Button(description='上传并下一条')
    submit_btn = widgets.Button(description='完成并处理')
    manual_out = widgets.Output()

    def on_next(b):
        with manual_out:
            # 保留先前信息
            rec = {
                '国号': field_guo.value.strip(),
                '帝号': field_di.value.strip(),
                '年号': field_era.value.strip(),
                '年': field_year.value.strip(),
                '月': field_month.value.strip(),
                '日': field_day.value.strip()
            }
            err = validate_entry(rec)
            if err:
                print(f"❌ {err}")
                return
            entries.append(rec)
            print(f"✅ 已添加 {rec['国号']}{rec['帝号']}{rec['年号']}{rec['年']}年{rec['月']}月{rec['日']}日")
            # 清空输入
            field_guo.value = ''
            field_di.value = ''
            field_era.value = ''
            field_year.value = ''
            field_month.value = ''
            field_day.value = ''

    def on_submit(b):
        with manual_out:
            if not entries:
                print("❌ 请先点击“下一条”添加至少一条记录。")
                return
            df = pd.DataFrame(entries)
            print("✅ 所有记录：")
            display(df)
            convert_and_download(df, 'manual_input.csv')

    next_btn.on_click(on_next)
    submit_btn.on_click(on_submit)
    display(
        widgets.VBox([
            widgets.HBox([field_guo, field_di, field_era]),  # 前三个输入框一行
            widgets.HBox([field_year, field_month, field_day]),  # 后三个输入框一行
            widgets.HBox([next_btn, submit_btn]),
            manual_out
        ])
    )

# 主界面：模式选择 + 确定按钮
mode_selector = widgets.RadioButtons(
    options=[('上传 Excel 文件', 'excel'), ('手动输入/粘贴中历日期', 'manual')],
    description='选择模式：'
)
confirm_button = widgets.Button(description='确定')
ui_output = widgets.Output()

confirm_button.on_click(lambda b: (ui_output.clear_output(), upload_excel_ui() if mode_selector.value=='excel' else manual_input_ui()))

display(mode_selector, confirm_button, ui_output)


---

### 📄 聲明與開發資訊 | Copyright & Credits

本工具由多倫多大學鄭裕彤東亞圖書館設計，程式碼版權歸本館所有。  
Developed by the Cheng Yu Tung East Asian Library, University of Toronto. Code copyright belongs to the Library.

西曆轉換結果依賴中央研究院數位文化中心所提供的中曆轉換系統。  
Gregorian date conversion results rely on the Sinocal system by Academia Sinica Center for Digital Cultures (ASCDC).  
  
    
🔗 東亞圖書館研究指南頁面 | EAL Research Guide: *[Add URL]*  
🔗 中央研究院中曆轉換系統 | Academia Sinica Sinocal Website:
<https://sinocal.sinica.edu.tw/>
